In [4]:
import urllib.parse, urllib.request
from bs4 import BeautifulSoup as bs

url = "http://www.google.com/finance"        

#open url and get its content
try:        
    htmltext = urllib.request.urlopen(url).read()
except Exception as ex:
    print(ex)

#declare data structures
sector = []
delta = []
movers = []
movers_change = []
i = 0
j = 0

#find table, then find <td> tags by style. 
#Based on style add the sector and its change to the respective data strucuture
soup = bs(htmltext)
table = soup.find("div", {"id" : "secperf"}).find("table")
for tr in table.findAll("tr"):
    for td in tr.findAll("td", {"style" : "text-align:left; padding-right:0.5em"}):
        if(td.text.strip() in 'Sector'): #we don't want the Sector in our list
            continue
        sector.append(td.text.strip())
    for td in tr.findAll("td", {"style" : "width:20%; padding-right:1em"}):
        delta.append(td.text.strip())

composite = (list(zip(delta, sector))) #zip the two lists to create a list that contains sector and change
max_delta = abs(float((max(composite, key = lambda item:item[0])[0]).strip("%"))) #find absolute max change
min_delta = abs(float((min(composite, key = lambda item:item[0])[0]).strip("%"))) #find absolute min change

if max_delta > min_delta: #check for the absolute maximum and set that as sector of interest
    delta_sector = max(composite, key = lambda item:item[0])
else:
    delta_sector = min(composite, key = lambda item:item[0])

for link in table.findAll('a', href = True, text = delta_sector[1]): #find the url of the sector
    childUrl = link['href']
   
new_url = urllib.parse.urljoin(url, childUrl) #join relative url to parent url

try: #open url and get its HTML       
    htmltext = urllib.request.urlopen(new_url).read()
except Exception as ex:
    print(ex)
   
soup = bs(htmltext)
table = soup.find("table", {"class" : "topmovers"}) #find the table topmovers
for tr in table.findAll("tr"):
    for td in tr.findAll("td"):
        if td.find("a") is not None: 
            i = i + 1
            if i == 1 or i == 11: #find all <a> tags and store only those corresponding to biggest gainer and loser
                movers.append(td.text.strip())
        if td.find("span") is not None:
            j = j + 1
            if i == 2 or i == 12: #find all <span> tags and store only those corresponding to biggest gainer and loser
                s = td.text.strip()
                movers_change.append(s[s.find("(") + 1 : s.find(")")]) #extract only numbers between parenthese which is the %change

movers = list(zip(movers, movers_change)) #zip the movers and changes together

#print the output
print("The sector that has moved the most is " + delta_sector[1] + " " + delta_sector[0] + ".")
print(movers[0][0] + " gained the most (" + movers[0][1] + ") while " + movers[1][0] + ", the biggest loser, lost " + movers[1][1])        
    
        

The sector that has moved the most is Utilities -3.51%.
AES Corp gained the most (2.43%) while South Jersey Industries, the biggest loser, lost -8.53%
